In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os
from datetime import datetime

import matplotlib.pyplot as plt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import ccxt

sys.path.append("..") 
from tp_config import *
from tp_utils.data_provider import read_prices, read_data,load_data_from_exchange
from portfolio_tools import (load_data_for_portfolio, calc_frontier,
                             calc_weights, weights_to_df, print_data)

## Helper functions

In [52]:
now = datetime.today().strftime('%Y-%m-%d')
def start_end_win(period, offset, horizon, lookback):
    start_win = offset + period * horizon
    end_win = start_win + lookback
    return (start_win, end_win)


In [143]:
data_path = DATA_PATH_CRYPTO

pair = 'USDT'
#pair = 'BTC'

index_assets = pd.read_csv('index_assets.csv')['asset'].tolist()
index_low_assets = pd.read_csv('index_low_assets.csv')['asset'].tolist()
assets = index_assets + index_low_assets
markets = [m + '-' + pair for m in assets]

### Data from exchange

In [145]:
exchange = ccxt.bybit()
start_timestamp = exchange.parse8601('2024-01-01 00:00:00')
timeframes = {'1h':60, '4h':240}

data_path = DATA_PATH_CRYPTO + '/' + pair
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, 10, verbose = False)

D:\Code\Projects\QuantMod\tp\Portfolio\..\tp_utils\data_provider.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])


Done
Time taken = 0.012  hours


## Read from local data

In [16]:
markets = [x for x in markets if x != 'FLT-USDT']

In [146]:
df_all =  load_data_for_portfolio(markets, '4h')   
print(datetime.fromtimestamp(df_all.index[0]/1000))
print(datetime.fromtimestamp(df_all.index[-1]/1000))

AVAX-USDT
BCH-USDT
BTC-USDT
BNB-USDT
DOT-USDT
DOGE-USDT
ETH-USDT
LINK-USDT
LTC-USDT
LUNA-USDT
MATIC-USDT
MNT-USDT
SOL-USDT
TON-USDT
TRX-USDT
AAVE-USDT
ADA-USDT
AEVO-USDT
ALGO-USDT
APT-USDT
AXS-USDT
BONK-USDT
BOME-USDT
CAKE-USDT
CYBER-USDT
CPOOL-USDT
EOS-USDT
ETC-USDT
ETHFI-USDT
FIRE-USDT
FLT-USDT
FTT-USDT
FTM-USDT
GALA-USDT
INJ-USDT
ICP-USDT
JUP-USDT
NEON-USDT
NEAR-USDT
ONDO-USDT
PEPE-USDT
PENDLE-USDT
PPT-USDT
PYTH-USDT
RUNE-USDT
SEI-USDT
SQT-USDT
SUI-USDT
THETA-USDT
TOKEN-USDT
UNI-USDT
WAVES-USDT
WLD-USDT
XRP-USDT
ZIL-USDT
2023-10-01 04:00:00
2024-03-28 20:00:00


In [147]:
since = 1703874400000
df_prices = df_all[df_all.index > since]

print(datetime.fromtimestamp(df_prices.index[0]/1000))
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

2023-12-30 00:00:00
2024-03-28 20:00:00


In [148]:
import riskfolio as rp


def riskfolio_weights(df_period, obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    rm = 'CVaR' # Risk measure used, this time will be variance
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 2
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

In [149]:
df_prices

,AVAX-USDT,BCH-USDT,BTC-USDT,BNB-USDT,DOT-USDT,DOGE-USDT,ETH-USDT,LINK-USDT,LTC-USDT,LUNA-USDT,...,SEI-USDT,SQT-USDT,SUI-USDT,THETA-USDT,TOKEN-USDT,UNI-USDT,WAVES-USDT,WLD-USDT,XRP-USDT,ZIL-USDT
T,,,,,,,,,,,,,,,,,,,,,
1703880000000,40.1956,254.1,42064.00,313.5979,8.289,0.09096,2299.19,15.5422,73.22,0.8532,...,0.5505,NaN,0.8058,1.2746,0.03491,7.5694,2.7841,3.5061,0.6228,0.02523
1703894400000,39.7363,254.6,41974.01,315.8525,8.362,0.09059,2298.18,15.2863,72.94,0.8520,...,0.5401,NaN,0.7910,1.3215,0.03465,7.6849,2.7560,3.5964,0.6204,0.02512
1703908800000,39.1181,284.4,41946.00,318.1244,8.318,0.09046,2295.60,15.3066,73.89,0.8670,...,0.5428,NaN,0.7916,1.3305,0.03437,7.6338,2.7450,3.5717,0.6221,0.02512
1703923200000,39.1695,278.8,41899.43,316.7286,8.206,0.08991,2287.80,15.1467,73.03,0.8372,...,0.5380,NaN,0.7764,1.2756,0.03375,7.4603,2.7009,3.5115,0.6205,0.02482
1703937600000,39.8057,277.3,42373.01,318.8640,8.383,0.09080,2312.90,15.3761,73.62,0.8559,...,0.5500,NaN,0.7891,1.2775,0.03396,7.5022,2.7549,3.9278,0.6280,0.02510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711584000000,53.7450,530.6,69295.11,581.7479,9.369,0.19481,3496.63,19.1899,95.19,1.0949,...,0.8713,0.04120,2.0149,2.9379,0.19125,12.2360,3.7263,8.3080,0.6093,0.03505
1711598400000,53.8068,565.2,69465.04,583.6873,9.397,0.19593,3501.80,19.2088,95.34,1.1000,...,0.8759,0.03974,2.0036,2.9475,0.19109,12.2680,3.7450,8.3408,0.6105,0.03521
1711612800000,54.3460,574.2,70643.52,587.2524,9.573,0.21717,3579.47,19.5936,95.08,1.1048,...,0.8622,0.04023,1.9920,2.9729,0.18142,12.3971,3.9081,8.4926,0.6213,0.03695


In [151]:

def index_date(ind):
    return datetime.fromtimestamp(ind/1000)

class Portfolio:
    def __init__(self, balance):
        self.balance   = balance
        self.portfolio = {}
        self.buy_correction = 1.001
        self.sell_correction = 0.99
        
    def sell_portfolio(self, prices):
        for key in self.portfolio.keys():
            asset_price = prices[key]
            self.balance = self.balance + self.portfolio[key] * asset_price * self.sell_correction
        self.portfolio = {}
        
    def buy_portfolio(self, portfolio,  prices):
        overall_balance = self.balance
        for key in portfolio.keys():
            asset_price = prices[key]
            sum_for_asset = portfolio[key] * overall_balance 
            self.balance = self.balance - sum_for_asset 
            quant = sum_for_asset / (asset_price * 1.00075)
            self.portfolio[key] = quant

In [100]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

In [158]:
lookback = 120
rebalance_period = 6
offset = 8
start_trade = lookback + offset

port = Portfolio(100)

count = 0 

for ind, row in df_prices.iloc[start_trade:].iterrows():
    if count % rebalance_period == 0:
        print('Rebalance data: ', ind, index_date(ind))
        
        port.sell_portfolio(row) 
        print("Баланс портфеля", port.balance)
        
        df_period = df_prices.loc[:ind]
        df_period = df_period.dropna(axis = 1)
        df_period = df_period[-lookback:]
      #  ef  = calc_frontier(df_period, "semicovariance",  "ema_historical_return", span = 150)
#        ef  = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
#        dfw = calc_weights(ef, 'max_sharpe', 0)      
        
        dfw = riskfolio_weights(df_period, "MaxRet")
        
        new_portfolio = dfw.to_dict()['weights']
        print("Новый портфель: ", new_portfolio)
        
        port.buy_portfolio(new_portfolio, row)
         
        print()
    count = count + 1

Rebalance data:  1705723200000 2024-01-20 08:00:00
Баланс портфеля 100
Новый портфель:  {'PENDLE-USDT': 0.6121775687532612, 'PPT-USDT': 0.3520811217183454, 'SUI-USDT': 0.035741231851173734}

Rebalance data:  1705809600000 2024-01-21 08:00:00
Баланс портфеля 103.58733204573505
Новый портфель:  {'PENDLE-USDT': 0.665308094671562, 'PPT-USDT': 0.19338350160696627, 'SUI-USDT': 0.14130785531888376}

Rebalance data:  1705896000000 2024-01-22 08:00:00
Баланс портфеля 102.71180247115225
Новый портфель:  {'PENDLE-USDT': 0.6020205955221591, 'PPT-USDT': 0.3698366080744508, 'SUI-USDT': 0.02814254904569273}

Rebalance data:  1705982400000 2024-01-23 08:00:00
Баланс портфеля 89.89023828383671
Новый портфель:  {'PENDLE-USDT': 0.6580817987609973, 'PPT-USDT': 0.23605235196969493, 'SUI-USDT': 0.10586506722477755}

Rebalance data:  1706068800000 2024-01-24 08:00:00
Баланс портфеля 93.74035228312476
Новый портфель:  {'PENDLE-USDT': 0.5637861510152954, 'SUI-USDT': 0.4266781128422182}

Rebalance data:  170615

In [33]:
lookback = 120 # 96, 95
horizon = 6 # 8, 10

res_trade = []
offset = 9 #np.random.randint(12)#,  8,9,10,11
n_periods = (df_prices.shape[0]  - lookback) // horizon
print('offset', offset)
balance = 100
portfolio = {}
for period in range(n_periods):

    start_win, end_win = start_end_win(period, offset, horizon, lookback)
  
    
#Calc frontier
    df_period = df_prices.iloc[start_win:end_win]
    df_period = df_period.dropna(axis = 1)
    print_data(df_period)
#    ef = calc_frontier(df_period, "semicovariance",  "ema_historical_return")
    ef = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
#    ef = calc_frontier(df_period, "sample_cov")
    dfw = calc_weights(ef, 'max_sharpe', 0)      
#    dfw = calc_weights(ef, 'efficient_risk', 0.7) 
    
    
#    dfw = riskfolio_weights(df_period, "Sharpe")

#Calc weights    
#    try:

        #dfw = calc_weights(ef, 'efficient_risk', 0.4) 
#        pass
#    except:

#continue
#    print(dfw)

#Buy portfolio    
    portfolio = dfw.to_dict()['weights']
    print(portfolio)
    
    if end_win < df_prices.shape[0] :
        prices = df_prices.iloc[end_win]
        buy_time = index_date(df_prices, end_win)
        portfolio, balance = buy_portfolio(prices, balance)

    # Sell portfolio      
        sell_index = end_win + horizon
        sell_index = min(sell_index, df_prices.shape[0] - 1)
        balance = sell_portfolio(portfolio, sell_index, balance)
        cur_time = index_date(df_prices, sell_index)
        print(period, 'Sell time', cur_time , balance)    

        res_trade.append(balance)

offset 9
2023-12-31 12:00:00
2024-01-20 08:00:00
{'PENDLE-USDT': 0.60166, 'PPT-USDT': 0.29005, 'PYTH-USDT': 0.10829}
0 Sell time 2024-01-21 12:00:00 110.09386900466522
2024-01-01 12:00:00
2024-01-21 08:00:00
{'PENDLE-USDT': 0.5682856828568286, 'PPT-USDT': 0.15965159651596517, 'PYTH-USDT': 0.2720627206272063}
1 Sell time 2024-01-22 12:00:00 100.98974184124786
2024-01-02 12:00:00
2024-01-22 08:00:00
{'PENDLE-USDT': 0.76678, 'PPT-USDT': 0.1353, 'PYTH-USDT': 0.09792}
2 Sell time 2024-01-23 12:00:00 90.00679735526194
2024-01-03 12:00:00
2024-01-23 08:00:00
{'PENDLE-USDT': 0.45456, 'PPT-USDT': 0.10112, 'PYTH-USDT': 0.44432}
3 Sell time 2024-01-24 12:00:00 98.97966993048345
2024-01-04 12:00:00
2024-01-24 08:00:00
{'PENDLE-USDT': 0.46882, 'PYTH-USDT': 0.10686, 'SUI-USDT': 0.42432}
4 Sell time 2024-01-25 12:00:00 101.99694093639826
2024-01-05 12:00:00
2024-01-25 08:00:00
{'PENDLE-USDT': 0.6891168911689116, 'PYTH-USDT': 0.03854038540385404, 'SUI-USDT': 0.2723427234272343}
5 Sell time 2024-01-26 

In [22]:
 df_prices.iloc[start_win:end_win]


,AVAX-USDT,BTC-USDT,BNB-USDT,DOT-USDT,DOGE-USDT,ETH-USDT,LINK-USDT,LTC-USDT,LUNA-USDT,MATIC-USDT,...,SEI-USDT,SQT-USDT,SUI-USDT,THETA-USDT,TOKEN-USDT,UNI-USDT,WAVES-USDT,WLD-USDT,XRP-USDT,ZIL-USDT
T,,,,,,,,,,,,,,,,,,,,,
1709884800000,43.5468,67743.98,474.0608,10.431,0.16890,3950.80,20.0004,87.90,1.1422,1.1584,...,0.9085,0.01515,1.5463,3.0511,0.08746,15.0458,3.3298,7.2805,0.6284,0.03458
1709899200000,42.3574,67614.50,472.7876,10.217,0.16345,3900.41,19.6896,86.46,1.0977,1.1177,...,0.8783,0.01497,1.5080,2.9992,0.09057,14.5152,3.2090,7.1855,0.6142,0.03347
1709913600000,42.5434,68869.12,484.4892,10.490,0.16768,3928.74,19.7510,87.01,1.1491,1.1281,...,0.8959,0.01566,1.5239,2.9761,0.08856,14.6107,3.3036,7.5098,0.6199,0.03410
1709928000000,42.6165,68108.02,485.7265,10.635,0.16454,3883.20,19.6928,88.34,1.1393,1.1247,...,0.9250,0.01515,1.5430,3.0452,0.08729,14.5991,3.3438,7.5272,0.6199,0.03489
1709942400000,42.5875,68232.70,489.1490,10.640,0.16908,3922.81,20.0458,88.83,1.1555,1.1371,...,0.9066,0.01501,1.5346,3.1349,0.09179,15.0036,3.3605,9.1280,0.6256,0.03534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711540800000,53.3232,68852.02,569.9740,9.379,0.18766,3513.81,19.1452,93.65,1.1000,1.0219,...,0.8554,0.03162,2.1055,2.9331,0.20755,12.3676,3.7982,8.5908,0.6109,0.03510
1711555200000,53.8515,68642.87,569.4129,9.413,0.18274,3491.44,19.2696,93.82,1.1021,1.0090,...,0.8674,0.02990,2.1379,2.9580,0.21202,12.2070,3.7934,8.4421,0.6143,0.03528
1711569600000,54.1103,69470.79,574.1989,9.442,0.19055,3502.29,19.2812,93.55,1.0984,1.0089,...,0.8931,0.03513,2.0750,2.9809,0.20191,12.2348,3.7922,8.4225,0.6119,0.03546


In [26]:
datetime.fromtimestamp(df_prices.index[end_win+4]/1000)

datetime.datetime(2021, 5, 23, 9, 0)

In [29]:
risk_methods = [
#    "ledoit_wolf_constant_variance",
    "ledoit_wolf",
#    "semicovariance"
]


return_methods = [
#    "mean_historical_return",
    "ema_historical_return",
    ]

## Mass cov

In [ ]:
%%time
res = []
offset = 0

for method in risk_methods: 
    print(method)
    for return_method in return_methods:
        for span in [300]:
            for lookback in range(82, 100):
                print(" ", lookback)
                for horizon in range(3,13):

                    n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
                    balance = 0.10
                    portfolio = {}
                    for period in range(n_periods):

                        start_win, end_win = start_end_win(period, offset, horizon, lookback)  

                    #Calc frontier
                        df_period = df_prices.iloc[start_win:end_win]
                        ef = calc_frontier(df_period, method,  return_method, span)
                        dfw = calc_weights(ef, 'max_sharpe', 0)      
                        #dfw = calc_weights(ef, 'efficient_risk', 0.35) 


                    #Buy portfolio    
                        portfolio = dfw.to_dict()['W']
                        #print(portfolio)
                        portfolio, balance = buy_portfolio(end_win, balance)

                    # Sell portfolio      
                        balance = sell_portfolio(portfolio, end_win + horizon, balance)
                        cur_time = datetime.fromtimestamp(df_prices.index[end_win+horizon]/1000)
                        #print(period, cur_time , balance)    

                    res.append([method, return_method, span, lookback, horizon, n_periods, balance])   

ledoit_wolf
  82


D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
D:\Soft\Anaconda\envs\t\Lib\

In [49]:
df = pd.DataFrame(res, columns = ['method', 'return_method', 'span', 'look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)
 
dfx.to_csv('index_01.csv', index = False)
#dfx = dfx[dfx['method'] != 'semicovariance']
#dfx.groupby('method').mean()
dfx[:20]

,method,return_method,span,look,horizon,n,balance
86,semicovariance,ema_historical_return,300,90,9,25,0.169693575
159,semicovariance,ema_historical_return,300,97,12,18,0.169340716
158,semicovariance,ema_historical_return,300,97,11,20,0.168435151
48,semicovariance,ema_historical_return,300,86,11,21,0.161159575
149,semicovariance,ema_historical_return,300,96,12,18,0.160226639
16,semicovariance,ema_historical_return,300,83,9,26,0.160004638
169,semicovariance,ema_historical_return,300,98,12,18,0.159507019
128,semicovariance,ema_historical_return,300,94,11,20,0.157484558
139,semicovariance,ema_historical_return,300,95,12,18,0.157375244
118,semicovariance,ema_historical_return,300,93,11,20,0.155565676


In [108]:
dfx.groupby(['look']).mean().sort_values('balance')

,span,horizon,n,balance
look,,,,
63,300.0,22.0,4.435897436,0.086803623
62,300.0,22.0,4.461538462,0.086866830
32,300.0,22.0,6.641025641,0.086943737
33,300.0,22.0,6.487179487,0.087032436
34,300.0,22.0,6.435897436,0.087556527
51,300.0,22.0,5.230769231,0.087705196
36,300.0,22.0,6.333333333,0.088396883
61,300.0,22.0,4.461538462,0.088443203
59,300.0,22.0,4.717948718,0.088556354


### Mass offset

In [284]:
res = []
offset = 0

for method in risk_methods: 
    print(method)
    for lookback in range(89, 92):
        print(" ", lookback)
        for horizon in [5,6,8, 9]:
            for offset in range(0,24):
                n_periods = (df_prices.shape[0]  - lookback - offset) // horizon - 1
                balance = 0.10
                portfolio = {}
                for period in range(n_periods):

                    start_win, end_win = start_end_win(period, offset, horizon, lookback)

                # Sell portfolio    

            #        print(balance)    
                    df_period = df_prices.iloc[start_win:end_win]
                    ef = calc_frontier(df_period, method)
                    try:
                        dfw = calc_weights(ef, 'max_sharpe', 0) 
                    except:
                        continue

                #Buy portfolio    
                    portfolio = dfw.to_dict()['W']
    #                print(portfolio)
                    portfolio, balance = buy_portfolio(end_win, balance)

                # Sell portfolio      
                    balance = sell_portfolio(portfolio, end_win + horizon, balance)
                    cur_time = datetime.fromtimestamp(df_prices.index[end_win+horizon]/1000)
    #                print(period, cur_time , balance)    


                    res.append([method, lookback, horizon, offset, n_periods, balance])        

semicovariance
  89
  90
  91
ledoit_wolf_constant_correlation
  89
  90
  91


In [292]:
df = pd.DataFrame(res, columns = ['method', 'look', 'horizon', 'offset','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx.groupby(['method','look', 'horizon' ]).mean().sort_values('balance')

offset             n  \
method                           look horizon                               
ledoit_wolf_constant_correlation 91   5        11.233065442  36.347876005   
                                 90   5        11.231735160  36.557077626   
                                      6        11.229281768  30.215469613   
                                      9        11.216560510  19.662420382   
                                 89   5        11.236095346  36.763904654   
                                 91   9        11.211538462  19.538461538   
                                      6        11.225000000  30.050000000   
                                      8        11.239171375  22.160075330   
                                 89   6        11.239010989  30.379120879   
                                 90   8        11.226591760  22.288389513   
                                 89   9        11.227848101  19.784810127   
semicovariance                   91   5        11.233065442  36.347876005   
ledoit_wolf_constant_correlation 89   8        11.219739292  22.415270019   
semicovariance                   91   6        11.225000000  30.050000000   
                                      9        11.211538462  19.538461538   
                                      8        11.239171375  22.160075330   
                                 90   6        11.229281768  30.215469613   
                                      9        11.216560510  19.662420382   
                                      5        11.231735160  36.557077626   
                                 89   5        11.236095346  36.763904654   
                                 90   8        11.226591760  22.288389513   
                                 89   9        11.227848101  19.784810127   
                                      6        11.239010989  30.379120879   
                                      8        11.219739292  22.415270019   

                                                   balance  
method                           look horizon               
ledoit_wolf_constant_correlation 91   5        0.127415394  
                                 90   5        0.127716651  
                                      6        0.127924961  
                                      9        0.127941528  
                                 89   5        0.127996540  
                                 91   9        0.128014798  
                                      6        0.128110338  
                                      8        0.128145928  
                                 89   6        0.128309196  
                                 90   8        0.128335053  
                                 89   9        0.128353634  
semicovariance                   91   5        0.128397841  
ledoit_wolf_constant_correlation 89   8        0.128462395  
semicovariance                   91   6        0.129114217  
                                      9        0.129166876  
                                      8        0.129203905  
                                 90   6        0.129546970  
                                      9        0.129672050  
                                      5        0.129681249  
                                 89   5        0.129838040  
                                 90   8        0.130105575  
                                 89   9        0.130506244  
                                      6        0.130511691  
                                      8        0.130532659

(291, 29)

In [274]:
df = pd.DataFrame(res, columns = ['method', 'look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[(dfx['look'] > 85) & (dfx['look'] < 92)]
dfx[:20]

,method,look,horizon,n,balance
1969,semicovariance,89,8,25,0.197796092
2139,semicovariance,90,9,22,0.193421789
2031,semicovariance,90,5,40,0.191268967
1767,semicovariance,88,6,34,0.190027756
2179,semicovariance,91,5,40,0.189497499
3578,semicovariance,100,9,21,0.188080246
3754,semicovariance,102,5,38,0.187885951
3755,semicovariance,102,5,38,0.187838461
1991,semicovariance,89,9,22,0.187554015
2387,semicovariance,92,7,28,0.187062521


In [228]:
dfx.groupby(['horizon']).mean().sort_values('balance')

,look,n,balance
horizon,,,
9,90.0,14.08,0.139155017
8,90.0,16.08,0.139321131
6,90.0,21.92,0.142085156
7,90.0,18.56,0.142215123
5,90.0,26.60,0.143235112


In [126]:
### Mass

In [ ]:
res = []
offset = 0

for lookback in range(72, 102):
    print(lookback)
    for horizon in range(5,11):

        n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
        balance = 0.10
        portfolio = {}
        all_balance = 0
        for period in range(n_periods):

            start_win, end_win = start_end_win(period, horizon, lookback)

        # Sell portfolio    

    #        print(balance)    
            df_period = df_prices.iloc[start_win:end_win]
            ef = calc_frontier(df_period, 'ledoit_wolf')
            try:
                dfw = calc_weights(ef, 'max_sharpe', 0) 
            except:
                continue

            balance = sell_portfolio(portfolio, end_win, balance)
            all_balance = balance
        #Buy portfolio    
            portfolio = dfw.to_dict()['W']
            portfolio, balance = buy_portfolio(end_win, balance)

        res.append([lookback, horizon, n_periods, all_balance])

In [ ]:
df = pd.DataFrame(res, columns = ['look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

In [230]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

In [ ]:
res = []
offset = 0
#0.35, 95, 10
for par in [0.35, 0.4, 0.45, 0.5]:
    for cov_method in risk_methods:
        for lookback in range(80, 100):
            print(lookback)
            for horizon in range(2,12):
                profit = 0
                n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
                balance = 0.1
                portfolio = {}
                all_balance = 0
                for period in range(n_periods):

                    start_win, end_win = start_end_win(period, horizon, lookback)

                # Sell portfolio    

            #        print(balance)    
                    df_period = df_prices.iloc[start_win:end_win]
                    ef = calc_frontier(df_period, cov_method)
                    dfw = calc_weights(ef, 'efficient_risk', par) 
                    try:
                        pass
                    except:
                        continue

                    balance = sell_portfolio(portfolio, end_win, balance)
                    all_balance = balance
                    profit = profit + balance - 0.1
                #Buy portfolio    
                    balance = 0.10
                    portfolio = dfw.to_dict()['W']
                    portfolio, balance = buy_portfolio(end_win, balance)

                res.append([par, cov_method, lookback, horizon, n_periods, all_balance, profit])

In [232]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20] 

,par,cov,look,horizon,n,balance,profit
2625,0.40,ledoit_wolf_single_factor,82,7,20,0.107254666,0.076422891
2825,0.40,ledoit_wolf_constant_correlation,82,7,20,0.107145394,0.076284547
2904,0.40,ledoit_wolf_constant_correlation,90,6,22,0.107917221,0.075631286
2104,0.40,exp_cov,90,6,22,0.107943484,0.075624305
1625,0.40,sample_cov,82,7,20,0.107233674,0.075388251
2704,0.40,ledoit_wolf_single_factor,90,6,22,0.107904749,0.075289034
2813,0.40,ledoit_wolf_constant_correlation,81,5,28,0.103346873,0.075159280
1704,0.40,sample_cov,90,6,22,0.107906898,0.074867175
2877,0.40,ledoit_wolf_constant_correlation,87,9,14,0.132459390,0.074714529
4425,0.45,ledoit_wolf_constant_correlation,82,7,20,0.108042789,0.074680226


In [71]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

,par,cov,look,horizon,n,balance,profit
199,0.37,ledoit_wolf,95,10,91,0.102490153,0.316876595
149,0.37,ledoit_wolf,90,10,92,0.088793456,0.289672315
158,0.37,ledoit_wolf,91,9,102,0.092925225,0.289150017
222,0.37,ledoit_wolf,98,3,307,0.110125612,0.288139501
138,0.37,ledoit_wolf,89,9,102,0.089232278,0.285607226
153,0.37,ledoit_wolf,91,4,232,0.107691926,0.284661077
148,0.37,ledoit_wolf,90,9,102,0.087458152,0.284210233
46,0.37,ledoit_wolf,80,7,133,0.097544911,0.284045753
215,0.37,ledoit_wolf,97,6,153,0.101016831,0.281120509
128,0.37,ledoit_wolf,88,9,103,0.095319928,0.280706351


In [67]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

,par,cov,look,horizon,n,balance,profit
199,0.35,ledoit_wolf,95,10,91,0.102404941,0.303427301
158,0.35,ledoit_wolf,91,9,102,0.092925225,0.278199292
222,0.35,ledoit_wolf,98,3,307,0.109691921,0.275417148
149,0.35,ledoit_wolf,90,10,92,0.088793456,0.274179610
138,0.35,ledoit_wolf,89,9,102,0.089232278,0.272283932
46,0.35,ledoit_wolf,80,7,133,0.097476395,0.271506323
148,0.35,ledoit_wolf,90,9,102,0.087458152,0.271206971
153,0.35,ledoit_wolf,91,4,232,0.107211998,0.270897136
128,0.35,ledoit_wolf,88,9,103,0.095319928,0.268818052
143,0.35,ledoit_wolf,90,4,232,0.102069821,0.267836961


In [47]:
portfolio = df_final.to_dict()['W']

buy_data = lookback
for key in portfolio.keys():
    asset_price = df_prices.iloc[buy_data][key]
    balance = balance - portfolio[key] * asset_price
    if balance < 0:
        print('Balance < 0')

for key in portfolio.keys():
    asset_price = df_prices.iloc[sell_data][key]
    balance = balance + portfolio[key] * asset_price
    if balance < 0:
        print('Balance < 0')

NameError: name 'buy_data' is not defined

## Basic

In [121]:
# calculate expected returns and sample covariance amtrix

avg_returns = expected_returns.mean_historical_return(df_prices)
cov_mat = risk_models.sample_cov(df_prices)
cov_mat;

In [11]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 3000)

Expected annual return: 225.7%
Annual volatility: 54.4%
Sharpe Ratio: 4.11


,W
LEND-USDT,660.0
REN-USDT,760.0
ADA-USDT,450.0
KAVA-USDT,1080.0


In [12]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 4000)

Expected annual return: -9.7%
Annual volatility: 26.3%
Sharpe Ratio: -0.45


,W
BTC-USDT,1850.0
BNB-USDT,2140.0
